In [3]:
import glob 
import os 
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np 
from datetime import datetime , date
from tqdm import tqdm

In [4]:
def create_data_cube( path_img_crop, intrument):
    """
    This function create a date cube SDO AIA

    Last modified: Ordoñez Araujo, FJ  19.08.2022
    ------inputs----
    path_img     : it's the path where is the SDO image
    path_img_crop: it's the path where is the cropped SDO image
    
    intrument: this variable could be 'AIA' or 'HMI'
    and is necesary to differences on headers 
    -----outputs----
    data cube where its components is:
    
    data_cubo    :  data cube builed with SDO image  cropped 
    data_time_utc:  data_OBS of whole image  in forma %Y-%m-%dT%H:%M:%S.%f
    _______________________________________________________________________________________________________________
    WARNING:, ALL matrixs in the vector (data cube) should has the same shape, in general 
    when we do  the crop using sunpy this this will not be.
    when all the arrays in the data cube do not have the same shape you will have 
    problems trying to convert the list to an array . The error (likes, ValueError: could not 
    broadcast input array from shape (453,452) into shape (453,)) you will get was 
    reported in 
    url : https://stackoverflow.com/questions/43977463/valueerror-could-not-broadcast-input-array-from-shape-224-224-3-into-shape-2
    ______________________________________________________________________________________________________________
    """
    intrument=intrument.lower()

    cubo_image_crop  = [] #cube of image crop
    date_OBS         = [] # data obs
    
    x_image_crop     = sorted(glob.glob(path_img_crop)) # of SDO image cropped

    #-----------------Creating a date cube------------------------
    """
     It necesary considere two case becasuse the DATE-OBS in HMI has 
     another format ('2018-04-12T15:45:31.700001Z') that  AIA 
     DATE-OBS ('2018-04-12T15:45:28.84') 
    """
    date_format = "%Y-%m-%dT%H:%M:%S.%f"
    
    if intrument=='hmi':
        print('|------Doing HMI cubic------|')
        for i in tqdm(range(0,len(x_image_crop),1)):
            cubo_image_crop.append(fits.open(x_image_crop[i])[0].data)
            time_1=fits.open(x_image_crop[i])[0].header['DATE-OBS'][0:-1]
            date_OBS.append(datetime.strptime(time_1, date_format))


    if intrument=='aia':
        print('|------Doing AIA cubic------|')
        for i in tqdm(range(0,len(x_image_crop),1)):
            cubo_image_crop.append(fits.open(x_image_crop[i])[0].data) 
            time_1 =fits.open(x_image_crop[i])[0].header['DATE-OBS']
            date_OBS.append(datetime.strptime(time_1, date_format))
    
    return np.array(cubo_image_crop), np.array(date_OBS) 
    #return cubo_image_crop, np.array(date_OBS) 

In [5]:
data_cube, time_vec = create_data_cube('/home/javier/Documents/MSc-program/SDO_DATA_ALIGNIING/SDO/171/crop_and_rotate/*.fits', 'aia')

plt.imshow(data_cube[0])
data_cube.shape

|------Doing AIA cubic------|


100%|██████████| 275/275 [00:01<00:00, 218.07it/s]


In [13]:
os.system('mkdir SDO_data_cubic')
saving_cubo_AIA_MAGT    = np.save('SDO_data_cubic/aia_304_cube'         , data_cube, allow_pickle=True)
saving_cubo_AIA_timeUTC = np.save('SDO_data_cubic/aia_304_cube_time_vec', time_vec ,  allow_pickle=True)        


##  Saving data cube of crop the SDO images without rotating

In [8]:
wl=[94,131,171,193,211,304,335,1600,1700]
# ------------------Remenber--------------------------- 
# path_img      : it's the path where is the SDO image
# path_img_crop : it's the path where is the cropped SDO image
os.system('mkdir SDO_data_cubic')
for i in range(len(wl)):
    path_1 = '/home/javier/Documents/MSc-program/SDO_DATA_ALIGNIING/SDO/{}/crop_and_rotate/*.fits'.format(wl[i])
    x , y  = create_data_cube(path_1, 'AIA')
    #plt.figure()
    #plt.imshow(x[0][0]),plt.colorbar()
    #plt.title('AIA {}'.format(wl[i]))
    np.save('SDO_data_cubic/cuboAIA{}'.format(wl[i])          , x , allow_pickle=True)
    np.save('SDO_data_cubic/cuboAIA_timeUTC_{}'.format(wl[i]) , y , allow_pickle=True)
# HMI DATA     
path_mag  = '/home/javier/Documents/MSc-program/SDO_DATA_ALIGNIING/SDO/{}'  
data_cuboHMI_MAGT, time_mag_hmi            = create_data_cube(path_mag.format("magt/crop/*.fits"), 'HMI')
saving_cuboHMI_MAGT         = np.save('SDO_data_cubic/cuboHMI_MAGT'          , data_cuboHMI_MAGT, allow_pickle=True)
saving_cuboHMI_MAGT_timeUTC = np.save('SDO_data_cubic/cuboHMI_MAGT_timeUTC'  , time_mag_hmi,  allow_pickle=True)

|------Doing AIA cubic------|


100%|██████████| 349/349 [00:01<00:00, 226.82it/s]
/tmp/ipykernel_49063/1722308967.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(cubo_image_crop), np.array(date_OBS)


|------Doing AIA cubic------|


100%|██████████| 275/275 [00:01<00:00, 222.50it/s]


|------Doing AIA cubic------|


...

100%|██████████| 137/137 [00:00<00:00, 226.02it/s]


|------Doing AIA cubic------|


100%|██████████| 136/136 [00:00<00:00, 225.27it/s]


|------Doing HMI cubic------|


100%|██████████| 73/73 [00:00<00:00, 284.00it/s]


In [4]:
load_cuboAIA304  = np.load('SDO_data_cubic/cuboAIA304.npy'  ,allow_pickle=True)
load_cuboAIA1600 = np.load('SDO_data_cubic/cuboAIA1600.npy' ,allow_pickle=True)
load_cuboAIA1700 = np.load('SDO_data_cubic/cuboAIA1700.npy' ,allow_pickle=True)
load_cuboAIAMAGT = np.load('SDO_data_cubic/cuboHMI_MAGT.npy' ,allow_pickle=True)
